In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import *
from sklearn.cluster import *
from sklearn.preprocessing import StandardScaler
from sklearn.utils.testing import ignore_warnings

np.random.seed(0)
n_samples = 1500
blobs = make_blobs(n_samples=n_samples, random_state=8)
X, y = make_blobs(n_samples=n_samples, random_state=170)
anisotropic = (np.dot(X, [[0.6, -0.6], [-0.4, 0.8]]), y)
varied = make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=170)
noisy_circles = make_circles(n_samples=n_samples, factor=.5, noise=.05)
noisy_moons = make_moons(n_samples=n_samples, noise=.05)
no_structure = np.random.rand(n_samples, 2), None
datasets = {
    "같은 크기의 원형": blobs, 
    "같은 크기의 타원형": anisotropic, 
    "다른 크기의 원형": varied, 
    "초승달": noisy_moons, 
    "동심원": noisy_circles, 
    "비구조화": no_structure
}


plt.figure(figsize=(11, 11))
plot_num = 1
for i, (data_name, (X, y)) in enumerate(datasets.items()):
    if data_name in ["초승달", "동심원"]:
        n_clusters = 2
    else:
        n_clusters = 3

    X = StandardScaler().fit_transform(X)

    two_means = MiniBatchKMeans(n_clusters=n_clusters)
    dbscan = DBSCAN(eps=0.15)
    spectral = SpectralClustering(n_clusters=n_clusters, affinity="nearest_neighbors")
    ward = AgglomerativeClustering(n_clusters=n_clusters)
    affinity_propagation = AffinityPropagation(damping=0.9, preference=-200)
    clustering_algorithms = (
        ('K-Means', two_means),
        ('DBSCAN', dbscan),
        ('Hierarchical Clustering', ward),
        ('Affinity Propagation', affinity_propagation),
        ('Spectral Clustering', spectral),
    )

    for j, (name, algorithm) in enumerate(clustering_algorithms):
        with ignore_warnings(category=UserWarning):
            algorithm.fit(X)
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)
        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i == 0:
            plt.title(name)
        if j == 0:
            plt.ylabel(data_name)
        colors = plt.cm.tab10(np.arange(20, dtype=int))
        plt.scatter(X[:, 0], X[:, 1], s=5, color=colors[y_pred])
        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plot_num += 1

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
# A = np.arange(2*3*4).reshape((2,3,4))
# B = np.arange(2*3*4).reshape((2,4,3))

np.dot(A,B)
# (2, 3, 2, 3)
# np.matmul(A,B).shape
# (2, 3, 3)

## 군집화
: 주어진 데이터 집합을 유사한 데이터들의 그룹으로 나누는 방법론으로,
예측문제와 딜리 특정한 독립/종속 변수의 구분도 럾고 학습을 위한 목푯값도 필요없는 비지도 학습이다.

[방법]
- K평균 군집화
- 디비스캔 군집화
- 유사도 전파 군집화
- 계층적 군집화
- 스펙트럴 군집화 

군집화 방법은 사용법과 모수가 서로 다르다.
K평균이나 스펙트럴 군집화는 군집의 개수를 미리 지정해야 한다. 반면, 디비스캔과 유사도 전파법은 미리 정할 필요가 없지만, 모형에 따라 특별한 모수를 지정해주어야 하는데 모수 값에 따라 군집화 개수가 달라질 수 있다.

각 군집화 방법마다 특성이 다르므로 원하는 목적과 데이터 유형에 맞게 사용해야 한다. 또한 지정된 모수의 값에 따라 성능이 달라질 수 있다.

#### 군집화 성능기준¶
군집화의 경우에는 분류문제와 달리 성능기준을 만들기 어렵다. 심지어는 원래 데이터가 어떻게 군집화되어 있었는지를 보여주는 정답(groundtruth)이 있는 경우도 마찬가지이다. 따라서 다양한 성능기준이 사용되고 있다. 다음의 군집화 성능기준의 예다.

\
*★일치행렬 : N개 데이터 집합에서 i,j 2개의 데이터를 선택했을 때, 두 데이터가 같은 군집에 속하면 1, 다른 군집에 속하면 0이라 하자. 이 값을 N*N 행렬 T로 나타낼 수 있다.

    T=1 (군집i=군집j)
    =0 (군집i!=군집j)

\


1) 조정 랜드지수(Adjusted Rand Index)


2) 조정 상호정보량 (Adjusted Mutual Information)

3) 실루엣계수 (Silhouette Coefficient)



###### 일치 행렬

In [ ]:
'''
ex)예를 들어 {0,1,2,3,4}라는 5개의 데이터 집합에서 
{0,1,2}와 {3,4}가 각각 같은 군집라면 행렬 T는 다음과 같다.
'''
groundtruth   = np.array([
[1, 1, 1, 0, 0],
[1, 1, 1, 0, 0],
[1, 1, 1, 0, 0],
[0, 0, 0, 1, 1],
[0, 0, 0, 1, 1],])

In [ ]:
'''
이제 군집화 결과를 같은 방법으로 행렬 C로 표시하자. 
만약 군집화이 정확하다면 이 행렬은 정답을 이용해서 만든 행렬과 거의 같은 값을 가져야 한다.
만약 군집화 결과 {0,1}과 {2,3,4}가 같은 군집라면 행렬 C는 다음과 같다.
'''
clustering = np.array([
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1],
])

In [ ]:
'''
이 두 행렬의 모든 원소에 대해 값이 같으면 1 다르면 0으로 
계산한 행렬을 일치행렬(incidence matrix)이라고 한다. 
즉 데이터 집합에서 만들수 있는 모든 데이터 쌍에 대해 
정답과 군집화 결과에서 동일한 값을 나타내면 1, 다르면 0이 된다.
R=1 (Tij=Cij)
 =0 (Tij!=Cij)

즉, 원래 정답에서 1번 데이터와 2번 데이터가 같은(다른) 군집인데 
군집화 결과에서도 같은(다른) 군집이라고 하면 R12=1이다.
위 예제에서 일치행렬을 구하면 다음과 같다.
'''
incidence=1*(groundtruth==clustering)
incidence


In [ ]:
'''
이 일치 행렬은 두 데이터의 순서를 포함하므로 대칭행렬이다. 
만약 데이터의 순서를 무시한다면 위 행렬에서 대각성분과 
아래쪽 비대각 성분은 제외한 위쪽 비대각 성분만을 고려해야 한다. 
위쪽 비대각 성분에서 1의 개수는 다음과 같아진다.

a=T에서 같은 군집에 있고 C에서도 같은 군집에 있는 데이터 쌍의 수 
b=T에서 다른 군집에 있고 C에서도 다른 군집에 있는 데이터 쌍의 수
일치행렬 위쪽 비대각 성분에서 1의 개수=a+b
'''
np.fill_diagonal(incidence,0)  # 대각성분 제외
a_plus_b=np.sum(incidence)/2  # 대칭행렬의 절반만 계산
a_plus_b

###### 랜드지수
가능한 모든 데이터 쌍의 개수에 대해 정답인 데이터 쌍의 개수의 비율로 정의
Rand INdex=(a+b)/nC2

###### 조정 랜드지수
랜드지수는 0부터 1까지의 값을 가지고 1이 가장 좋은 성능을 뜻한다. 랜드지수의 문제점은 무작위로 군집화을 한 경우에도 어느 정도 좋은 값이 나올 가능성이 높다는 점이다. 즉 무작위 군집화에서 생기는 랜드지수의 기댓값이 너무 크다. 

이를 해결하기 위해 무작위 군집화에서 생기는 랜드지수의 기댓값을 원래의 값에서 빼서 기댓값과 분산을 재조정한 것이 조정 랜드지수(adjusted Rand index, ARI)다.

ARI=(RI-E[RI])/(max(RI)-E[RI])


In [ ]:
from scipy.special import comb
rand_index=a_plus_b/comb(incidence.shape[0],2) 
#incidence행렬 : 5X5 >> incidence.shape[0]=5
rand_index
'''
adjusted Rand index는 성능이 완벽한 경우 1이 된다. 
반대로 가장 나쁜 경우로서 무작위 군집화을 하면 0에 가까운 값이 나온다.
경우에 따라서는 음수가 나올 수도 있다.
'''

In [ ]:
'''
여러가지 군집화 방법을 적용하였을때 조정 랜드지수를 계산해보면
디비스캔과 스펙트럴 군집화의 값이 높게 나오는 것을 확인할 수 있다. 
scikit-learn 패키지의 metrics.cluster 서브패키지는 조정 랜드지수를 계산하는 adjusted_rand_score 명령을 제공한다.
'''
from sklearn.metrics.cluster import adjusted_rand_score

X,Y_true=anisotropic
X,Y_true
plot_num=1
X=StandardScaler().fit_transform(X)
for name,algorithm in clustering_algorithms:
    with ignore_warnings(category=UserWarning):
        algorithm.fit(X)
    if hasattr(algorithm,'labels_'):
        Y_pred=algorithm.labels_.astype(np.int)
    else:
        Y_pred=algorithm.predict(X)
    title="ARI={:5.3f}".format(adjusted_rand_score(Y_true,Y_pred))

In [ ]:
# hasattr?
# Return whether the object has an attribute with the given name.

###### 상호 정보량
두 확률 변수 간의 상호 의존성을 측정한 값.
군집화 결과를 이산확률변수로 가정
정답은 T={T1,T2,...,Tr}의 r개의 값을 가지는 이산확률변수이고
군집화 결과는 C={C1,C2,...,Cs}의 s개의 값을 가질 수 있는 이산확률변수라고 하자.

전체 데이터의 개수를 N이라고 하면 이산확률변수 T의 분포는
P(i)=|Ti|/N 로 추정할 수 있다. 이 식에서 |Ti|는 군집 Ti에 속하는 데이터의 개수를 나타낸다.

비슷하게 이산확률변수 C의 분포는
P'(j)=|Cj|/N 으로 추정하고 T와 C의 결합확률분포는 P(i,j)=|Ti∩Cj|/N로 추정한다. 여기서 |Ti∩Cj|는 군집 Ti에도 속하고 군집 Cj에도 속하는 데이터의 개수를 나타낸다.

확률변수 T,C의 상호 정보량은 MI(T,C)=∑i=1∑j=1(P(i,j)*{log(P(i,j)/P(i)P(j))}으로 정의한다.

만약 두 확률변수가 서로 독립이면 상호정보량의 값은 0이면 이 값이 상호정보량이 가질 수 있는 최소값이다. 두 확률변수가 의존성이 강할 수록 상호정보량이 증가한다. 또한 군집의 개수가 많아져도 상호정보량이 증가하므로 올바른 비교가 어렵다. 따라서 상호정보량의 기댓값을 빼서 재조정한 것이 조정 상호 정보량이다.


In [ ]:
# 상호정보량 & 조정상호정보량
from sklearn.metrics.cluster import adjusted_mutual_info_score

plt.figure(figsize=(12,2))
plot_num=1
for name,algorithm in clustering_algorithms:
    with ignore_warnings(category=UserWarning):
        algorithm.fit(X)
    if hasattr(algorithm,'labels_'):
        Y_pred=algorithm.labels_.astype(np.int)
    else:
        Y_pred=algorithm.predict(X)
    title="ARI={:5.3f}".format(adjusted_mutual_info_score(Y_true,Y_pred))
    
    plt.subplot(1,len(datasets),plot_num)
    plt.scatter(X[:,0],X[:,1],s=5,color=colors[y_pred])
    plt.xlim(-2.5,2.5)
    plt.ylim(-2.5,2.5)
    plt.xticks(())
    plt.yticks(())
    plt.title(title)
    plot_num+=1

###### 3) 실루엣 계수
원래 데이터가 속한 군집을 모를 경우 군집화 성능을 평가하는 지수.

모든 데이터 쌍(i,j)에 대해 거리/비윳도를 구한 다음, 이 결과를 이용하여 모든 데이터 i에 대해 다음 값을 구한다.
- ai : i와 같은 군집에 속한 원소들의 평균 거리
- bi : i와 다른 군집 중 가장 가까운 군집까지의 평균 거리

si=(bi−ai)/max(ai,bi)로 정의하고, 전체 데이터의 실루엣 계수의 평균된 값을 평균 실루엣계수라고 한다.

- 데이터 i에 대해 같은 군집의 데이터가 다른 군집의 데이터보다 더 가까운 경우 : 해당 데이터 (+)
- 다른 군집의 데이터가 같은 군집의 데이터보다 더 가깝다면 : 해당 데이터의 실루엣계수 (-) >>> 군집화이 잘못된 경우

잘못된 군집화에서는 실루엣계수가 음수인 데이터가 많아지므로 평균 실루엣계수가 작아진다. 따라서 실루엣계수가 클수록 좋은 군집화이라고 할 수 있다.

In [ ]:
# 실루엣 계수
from sklearn.metrics import silhouette_samples

def plot_silhouette(data):
    X=StandardScaler().fit_transform(data[0])
    colors=plt.cm.tab10(np.arange(20,dtype=int))
    plt.figure(figsize=(6,8))
    for i in range(4):
        model=SpectralClustering(n_clusters=i+2,affinity="nearest_neighbors")
        cluster_labels=model.fit_predict(X)
        sample_silhouette_values=silhouette_samples(X,cluster_labels)
        silhouette_avg=sample_silhouette_values.mean()

        plt.subplot(4,2,2*i+1)
        Y_lower=10
        for j in range(i+2):
            jth_cluster_sillhouette_values=sample_silhouette_values[cluster_labels==j]
            jth_cluster_sillhouette_values.sort()
            size_cluster_j=jth_cluster_sillhouette_values.shape[0]
            Y_upper=Y_lower+size_cluster_j
            plt.fill_betweenx(np.arange(Y_lower,Y_upper),
                              0,jth_cluster_sillhouette_values,
                              facecolor=colors[j],edgecolor=colors[j])
            plt.text(-0.05, Y_lower + 0.5 * size_cluster_j, str(j + 1))
            plt.axvline(x=silhouette_avg, color="red", linestyle="--")
            plt.xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
            plt.yticks([])
            plt.title("Sillhouette Average: {:5.2f}".format(silhouette_avg))
            Y_lower = Y_upper + 10

        plt.subplot(4, 2, 2 * i + 2)
        plt.scatter(X[:, 0], X[:, 1], s=5, color=colors[cluster_labels])
        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.title("Num. of Clusters : {}".format(i + 2))

    plt.tight_layout()
    plt.show()
    
plot_silhouette(blobs)